In [1]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [2]:
import PrjtCF_module as cf

# Input Index Data

In [3]:
# 기간 index 설정
idx = cf.PrjtIndex(idxname=['prjt', 'loan'], 
                   start=['2021-01', '2021-02'], 
                   periods=[24+1, 20+1], 
                   freq='M')

# 공정률 index 설정
idx.prcs = Series(np.ones(len(idx)) / len(idx), index=idx.index)

# Input Sales Data

In [4]:
# Initialize Sales Data
# Product A : 상온창고
prdtA = cf.Account(idx, "ProductA", ("Sales", "Prdt"))
prdtA.rent = 21_000 # 월 임대료 21,000원/평 가정
prdtA.area = 8_000 # 면적 8,000평
prdtA.cap = 0.053 # Cap rate 5.3%
prdtA.amt = prdtA.rent * 12 * prdtA.area / prdtA.cap / 1_000_000
prdtA.addamt(idx[0], prdtA.amt)
prdtA.subscdd(idx[20], prdtA.amt)

# Initialize Sales Data
# Product B : 저온창고
prdtB = cf.Account(idx, "ProductB", ("Prdt", "B"))
prdtB.rent = 50_000 # 월 임대료 50,000원/평 가정
prdtB.area = 6_500 # 면적 6,500평
prdtB.cap = 0.058 # Cap rate 5.8%
prdtB.amt = prdtB.rent * 12 * prdtB.area / prdtB.cap / 1_000_000
prdtB.addamt(idx[0], prdtA.amt)
prdtB.subscdd(idx[20], prdtB.amt)

# Merge
sales = cf.Merge({'prdtA':prdtA, 'prdtB':prdtB})

# Input Cost Data

In [5]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx, "Cost_Land", ("Cost", "Land"))
lnd.amt = 15_500.000
lnd.addscdd(idx[1], lnd.amt)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx, "Cost_Construction", ("Cost", "Construction"))
cstrn.amt = 50_000.000 # 도급공사비
cstrn.addscdd(idx.index, cstrn.amt * idx.prcs)
dct_cost['cstrn'] = cstrn

# 세금: 준공시 1회 지급
tax = cf.Account(idx, "Cost_Tax", ("Cost", "Tax"))
tax.amt = 3_000.000 # 각종 취득 제세금
tax.addscdd(idx[20], tax.amt)
dct_cost['tax'] = tax

# 운영비: 매월 1회 지급
oprtg = cf.Account(idx, "Cost_Operation", ("Cost", "Operating"))
oprtg.amt = 2_200.000
oprtg.addscdd(idx.index, np.ones(len(idx)) * oprtg.amt)
dct_cost['oprtg'] = oprtg

# Merge
cost = cf.Merge(dct_cost)

# Input Loan Data

In [6]:
dct_loan = {}

tra = cf.Loan(idx.loan, amt_ntnl=60_000, rate_fee=0.015,
              rate_IR=0.05, title="Tr.A", tag=("loan", "A"))
dct_loan['tra'] = tra

trb = cf.Loan(idx.loan, amt_ntnl=20_000, rate_fee=0.03,
              rate_IR=0.07, title="Tr.B", tag=("loan", "B"))
dct_loan['trb'] = trb

loan = cf.Merge(dct_loan)

# Cash Flow

In [7]:
# Make accounts
acc_oprtg = cf.Account(idx, "Operating Account", ("Account", "Operating"))

In [8]:
# Generating Unit Sales
def make_sales(idxno):
    sales['prdtA'].send(idxno, sales['prdtA'].sub_scdd[idxno], acc_oprtg)
    sales['prdtB'].send(idxno, sales['prdtB'].sub_scdd[idxno], acc_oprtg)

In [9]:
# Generating Unit Loan Notional Amt
def pay_loan_ntnl(idxno):
    if idxno in idx.loan.index:
    
        # pay notional amount of loan tra
        amt_send_a = loan['tra'].ntnl.sub_scdd[idxno]
        loan['tra'].ntnl.send(idxno, amt_send_a, acc_oprtg)
    
        # pay notional amount of loan tra
        amt_send_b = loan['trb'].ntnl.sub_scdd[idxno]
        loan['trb'].ntnl.send(idxno, amt_send_b, acc_oprtg)

In [10]:
# Generating Unit Cost
def make_cost(idxno):
    for name, cst in cost.dct.items():
        amt_send = cst.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_send, cst)

In [11]:
# Paying Financial Fee and IR
def pay_fntlcst(idxno):
    if idxno in idx.loan.index:
        # pay tra fee and IR
        amt_fee_a = loan['tra'].fee.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_fee_a, loan['tra'].fee)
        
        amt_IR_a = loan['tra'].IR.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_IR_a, loan['tra'].IR)
        
        amt_ntnl_a = loan['tra'].ntnl.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_ntnl_a, loan['tra'].ntnl)
        
        # pay trb fee and IR
        amt_fee_b = loan['trb'].fee.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_fee_b, loan['trb'].fee)
        
        amt_IR_b = loan['trb'].IR.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_IR_b, loan['trb'].IR)
        
        amt_ntnl_b = loan['trb'].ntnl.add_scdd[idxno]
        acc_oprtg.send(idxno, amt_ntnl_b, loan['trb'].ntnl)

In [12]:
# Make Cash Flow
for idxno in idx.index:
    make_sales(idxno)
    pay_loan_ntnl(idxno)
    make_cost(idxno)
    pay_fntlcst(idxno)

In [13]:
# Merge Accounts
dct_acc = {"sales":sales, "cost":cost, "loan":loan, "oprtg":acc_oprtg}
accttl = cf.Merge(dct_acc)

In [24]:
accttl.dfcol(['bal_strt', 'amt_add', 'amt_sub', 'bal_end'])

bal_strt                                         amt_add  \
                   sales      cost     loan          oprtg         sales   
2021-01-31      0.000000       0.0      0.0       0.000000  76075.471698   
2021-02-28  76075.471698    4200.0      0.0   -4200.000000      0.000000   
2021-03-31  76075.471698   23900.0 -78500.0   54600.000000      0.000000   
2021-04-30  76075.471698   28100.0 -74100.0   46000.000000      0.000000   
2021-05-31  76075.471698   32300.0 -69700.0   37400.000000      0.000000   
2021-06-30  76075.471698   36500.0 -65300.0   28800.000000      0.000000   
2021-07-31  76075.471698   40700.0 -60900.0   20200.000000      0.000000   
2021-08-31  76075.471698   44900.0 -56500.0   11600.000000      0.000000   
2021-09-30  76075.471698   49100.0 -52100.0    3000.000000      0.000000   
2021-10-31  76075.471698   53300.0 -47700.0   -5600.000000      0.000000   
2021-11-30  76075.471698   57500.0 -43300.0  -14200.000000      0.000000   
2021-12-31  76075.471698   61700.0 -38900.0  -22800.000000      0.000000   
2022-01-31  76075.471698   65900.0 -34500.0  -31400.000000      0.000000   
2022-02-28  76075.471698   70100.0 -30100.0  -40000.000000      0.000000   
2022-03-31  76075.471698   74300.0 -25700.0  -48600.000000      0.000000   
2022-04-30  76075.471698   78500.0 -21300.0  -57200.000000      0.000000   
2022-05-31  76075.471698   82700.0 -16900.0  -65800.000000      0.000000   
2022-06-30  76075.471698   86900.0 -12500.0  -74400.000000      0.000000   
2022-07-31  76075.471698   91100.0  -8100.0  -83000.000000      0.000000   
2022-08-31  76075.471698   95300.0  -3700.0  -91600.000000      0.000000   
2022-09-30  76075.471698   99500.0    700.0 -100200.000000      0.000000   
2022-10-31 -29203.643461  106700.0   5100.0   -6520.884841      0.000000   
2022-11-30 -29203.643461  110900.0      0.0  -95120.884841      0.000000   
2022-12-31 -29203.643461  115100.0      0.0  -99320.884841      0.000000   
2023-01-31 -29203.643461  119300.0      0.0 -103520.884841      0.000000   

                                                   amt_sub                \
               cost     loan          oprtg          sales cost     loan   
2021-01-31   4200.0      0.0       0.000000       0.000000  0.0      0.0   
2021-02-28  19700.0   1500.0   80000.000000       0.000000  0.0  80000.0   
2021-03-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-04-30   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-05-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-06-30   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-07-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-08-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-09-30   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-10-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-11-30   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2021-12-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-01-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-02-28   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-03-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-04-30   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-05-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-06-30   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-07-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-08-31   4200.0   4400.0       0.000000       0.000000  0.0      0.0   
2022-09-30   7200.0   4400.0  105279.115159  105279.115159  0.0      0.0   
2022-10-31   4200.0  84400.0       0.000000       0.000000  0.0      0.0   
2022-11-30   4200.0      0.0       0.000000       0.000000  0.0      0.0   
2022-12-31   4200.0      0.0       0.000000       0.000000  0.0